# Exercise07 : Hyperparameter Tuning

AML provides framework-independent hyperparameter tuning capability.    
This capability monitors accuracy in AML logs.

*back to [index](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/)*

## Variable's Setting

Replace below's branket's string and set the required variables.

In [1]:
my_resource_group = "{AML-RESOURCE-GROUP-NAME}"
my_workspace = "{AML-WORSPACE-NAME}"

## Save your training code

First, you must save your training code.    
Here we should use the source code in "[Exercise06 : Experimentation Logs and Outputs](./exercise06_experimentation.ipynb)", which sends logs periodically into AML run history.

Create ```scirpt``` directory.

In [2]:
import os
script_folder = './script'
os.makedirs(script_folder, exist_ok=True)

Save source code as ```./script/train_expriment.py```.

In [3]:
%%writefile script/train_experiment.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import os
import shutil
import argparse
import math

import tensorflow as tf

from azureml.core.run import Run

# Get run when running in remote
if 'run' not in locals():
    run = Run.get_context()

FLAGS = None
batch_size = 100

#
# define functions for Estimator
#

def _my_input_fn(filepath, num_epochs):
    # image - 784 (=28 x 28) elements of grey-scaled integer value [0, 1]
    # label - digit (0, 1, ..., 9)
    data_queue = tf.train.string_input_producer(
        [filepath],
        num_epochs = num_epochs) # data is repeated and it raises OutOfRange when data is over
    data_reader = tf.TFRecordReader()
    _, serialized_exam = data_reader.read(data_queue)
    data_exam = tf.parse_single_example(
        serialized_exam,
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        })
    data_image = tf.decode_raw(data_exam['image_raw'], tf.uint8)
    data_image.set_shape([784])
    data_image = tf.cast(data_image, tf.float32) * (1. / 255)
    data_label = tf.cast(data_exam['label'], tf.int32)
    data_batch_image, data_batch_label = tf.train.batch(
        [data_image, data_label],
        batch_size=batch_size)
    return {'inputs': data_batch_image}, data_batch_label

def _get_input_fn(filepath, num_epochs):
    return lambda: _my_input_fn(filepath, num_epochs)

def _my_model_fn(features, labels, mode):
    # with tf.device(...): # You can set device if using GPUs

    # define network and inference
    # (simple 2 fully connected hidden layer : 784->128->64->10)
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
            tf.truncated_normal(
                [784, FLAGS.first_layer],
                stddev=1.0 / math.sqrt(float(784))),
            name='weights')
        biases = tf.Variable(
            tf.zeros([FLAGS.first_layer]),
            name='biases')
        hidden1 = tf.nn.relu(tf.matmul(features['inputs'], weights) + biases)
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal(
                [FLAGS.first_layer, FLAGS.second_layer],
                stddev=1.0 / math.sqrt(float(FLAGS.first_layer))),
            name='weights')
        biases = tf.Variable(
            tf.zeros([FLAGS.second_layer]),
            name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal(
                [FLAGS.second_layer, 10],
                stddev=1.0 / math.sqrt(float(FLAGS.second_layer))),
        name='weights')
        biases = tf.Variable(
            tf.zeros([10]),
            name='biases')
        logits = tf.matmul(hidden2, weights) + biases
 
    # compute evaluation matrix
    predicted_indices = tf.argmax(input=logits, axis=1)
    if mode != tf.estimator.ModeKeys.PREDICT:
        label_indices = tf.cast(labels, tf.int32)
        accuracy = tf.metrics.accuracy(label_indices, predicted_indices)
        tf.summary.scalar('accuracy', accuracy[1]) # output to TensorBoard 
        loss = tf.losses.sparse_softmax_cross_entropy(
            labels=labels,
            logits=logits)
 
    # define operations
    if mode == tf.estimator.ModeKeys.TRAIN:
        #global_step = tf.train.create_global_step()
        #global_step = tf.contrib.framework.get_or_create_global_step()
        global_step = tf.train.get_or_create_global_step()        
        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=FLAGS.learning_rate)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=global_step)
        # Ask for accuracy and loss in each steps
        class _CustomLoggingHook(tf.train.SessionRunHook):
            def begin(self):
                self.training_accuracy = []
                self.training_loss = []
            def before_run(self, run_context):
                return tf.train.SessionRunArgs([accuracy[1], loss, global_step])
            def after_run(self, run_context, run_values):
                result_accuracy, result_loss, result_step = run_values.results
                if result_step % 10 == 0 :
                    self.training_accuracy.append(result_accuracy)
                    self.training_loss.append(result_loss)
                if result_step % 100 == 0 : # save logs in each 100 steps
                    run.log_list('training_accuracy', self.training_accuracy)
                    run.log_list('training_loss', self.training_loss)
                    self.training_accuracy = []
                    self.training_loss = []
        return tf.estimator.EstimatorSpec(
            mode,
            training_chief_hooks=[_CustomLoggingHook()],
            loss=loss,
            train_op=train_op)
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            'accuracy': accuracy
        }
        return tf.estimator.EstimatorSpec(
            mode,
            loss=loss,
            eval_metric_ops=eval_metric_ops)
    if mode == tf.estimator.ModeKeys.PREDICT:
        probabilities = tf.nn.softmax(logits, name='softmax_tensor')
        predictions = {
            'classes': predicted_indices,
            'probabilities': probabilities
        }
        export_outputs = {
            'prediction': tf.estimator.export.PredictOutput(predictions)
        }
        return tf.estimator.EstimatorSpec(
            mode,
            predictions=predictions,
            export_outputs=export_outputs)

def _my_serving_input_fn():
    inputs = {'inputs': tf.placeholder(tf.float32, [None, 784])}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

#
# Main
#

parser = argparse.ArgumentParser()
parser.add_argument(
    '--data_folder',
    type=str,
    default='./data',
    help='Folder path for input data')
parser.add_argument(
    '--chkpoint_folder',
    type=str,
    default='./logs',  # AML experiments logs folder
    help='Folder path for checkpoint files')
parser.add_argument(
    '--model_folder',
    type=str,
    default='./outputs',  # AML experiments outputs folder
    help='Folder path for model output')
parser.add_argument(
    '--learning_rate',
    type=float,
    default='0.07',
    help='Learning Rate')
parser.add_argument(
    '--first_layer',
    type=int,
    default='128',
    help='Neuron number for the first hidden layer')
parser.add_argument(
    '--second_layer',
    type=int,
    default='64',
    help='Neuron number for the second hidden layer')
FLAGS, unparsed = parser.parse_known_args()

# clean checkpoint and model folder if exists
if os.path.exists(FLAGS.chkpoint_folder) :
    for file_name in os.listdir(FLAGS.chkpoint_folder):
        file_path = os.path.join(FLAGS.chkpoint_folder, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
if os.path.exists(FLAGS.model_folder) :
    for file_name in os.listdir(FLAGS.model_folder):
        file_path = os.path.join(FLAGS.model_folder, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)

# read TF_CONFIG
run_config = tf.estimator.RunConfig()

# create Estimator
mnist_fullyconnected_classifier = tf.estimator.Estimator(
    model_fn=_my_model_fn,
    model_dir=FLAGS.chkpoint_folder,
    config=run_config)
train_spec = tf.estimator.TrainSpec(
    input_fn=_get_input_fn(os.path.join(FLAGS.data_folder, 'train.tfrecords'), 2),
    max_steps=60000 * 2 / batch_size)
eval_spec = tf.estimator.EvalSpec(
    input_fn=_get_input_fn(os.path.join(FLAGS.data_folder, 'test.tfrecords'), 1),
    steps=10000 * 1 / batch_size,
    start_delay_secs=0)

# run !
eval_res = tf.estimator.train_and_evaluate(
    mnist_fullyconnected_classifier,
    train_spec,
    eval_spec
)

# save model and variables
model_dir = mnist_fullyconnected_classifier.export_savedmodel(
    export_dir_base = FLAGS.model_folder,
    serving_input_receiver_fn = _my_serving_input_fn)
print('current working directory is ', os.getcwd())
print('model is saved ', model_dir)

# send logs to AML
run.log('learning_rate', FLAGS.learning_rate)
run.log('1st_layer', FLAGS.first_layer)
run.log('2nd_layer', FLAGS.second_layer)
run.log('final_accuracy', eval_res[0]['accuracy'])
run.log('final_loss', eval_res[0]['loss'])

Overwriting script/train_experiment.py


## Create AML compute

Create AML compute pool for computing environment.

In [5]:
!az ml compute create --name hypertest01 \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --type amlcompute \
  --min-instances 0 \
  --max-instances 4 \
  --size Standard_D2_v2

Command group 'ml compute' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/computes/hypertest01",
  "idle_time_before_scale_down": 120,
  "location": "eastus",
  "max_instances": 4,
  "min_instances": 0,
  "name": "hypertest01",
  "network_settings": {},
  "provisioning_state": "Succeeded",
  "resourceGroup": "AML-rg",
  "size": "STANDARD_D2_V2",
  "ssh_public_access_enabled": true,
  "tier": "dedicated",
  "type": "amlcompute"
}


## Submit a job with hyper-parameter's search

Now submit a job, in which multiple trainings will run depending on different hyper-parameters.<br>
In this example, we monitor training accuracy depending on 3 arguments - ```--learning_rate```, ```--first_layer```, and ```--second_layer```. Each arguments can have 3 different values (and then total 27 trials can be run), but here I set 20 trials to run, in which the values of arguments are randomly picked up.
<br>These trials will be parralelized on above 4 node to speed up.

For ```sampling_algorithm```, you can use ```grid```, ```random```, and ```bayesian```.<br>
You can also specify an early termnination policy (```early_termination```), in which the training will terminate if the primary metric falls outside of some threshold.

> Note : In this example, I also use the registered dataset  (train.tfrecords, test.tfrecords) named ```mnist_tfrecords_dataset``` to mount in your compute target. Run "[Exercise02 : Prepare Data](./exercise02_prepare_data.ipynb)" for dataset preparation.

In [6]:
%%writefile 07_hyperparam_job.yml
$schema: https://azuremlschemas.azureedge.net/latest/sweepJob.schema.json
type: sweep
trial:
  code: 
    local_path: script
  command: >-
    python train_experiment.py
    --data_folder ${{inputs.mnist_tf}}
    --learning_rate ${{search_space.learning_rate}}
    --first_layer ${{search_space.first_layer}}
    --second_layer ${{search_space.second_layer}}
  environment: azureml:AzureML-TensorFlow-1.13-CPU:30
inputs:
  mnist_tf: 
    dataset: azureml:mnist_tfrecords_dataset:1
compute: azureml:hypertest01
sampling_algorithm: random
search_space:
  learning_rate:
    type: choice
    values: [0.01, 0.05, 0.9]
  first_layer:
    type: choice
    values: [100, 125, 150]
  second_layer:
    type: choice
    values: [30, 60, 90]
objective:
  goal: maximize
  primary_metric: training_accuracy
limits:
  max_total_trials: 20
  max_concurrent_trials: 4
display_name: hyperdrive_test
experiment_name: hyperdrive_test
description: This is example

Writing 07_hyperparam_job.yml


In [7]:
!az ml job create --file 07_hyperparam_job.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml job' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Uploading script (0.02 MBs): 100%|████| 23198/23198 [00:00<00:00, 839809.29it/s]


{
  "compute": "azureml:hypertest01",
  "creation_context": {
    "created_at": "2022-02-28T00:58:51.904469+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User"
  },
  "description": "This is example",
  "display_name": "hyperdrive_test",
  "experiment_name": "hyperdrive_test",
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/jobs/abae7524-ff7c-42dc-802c-a23eee7c44db",
  "inputs": {
    "mnist_tf": {
      "dataset": "azureml:mnist_tfrecords_dataset:1",
      "mode": "ro_mount"
    }
  },
  "limits": {
    "max_concurrent_trials": 4,
    "max_total_trials": 20,
    "timeout": 5184000
  },
  "name": "abae7524-ff7c-42dc-802c-a23eee7c44db",
  "objective": {
    "goa

## View logs

You can view logs and metrics in Experiments on [Azure ML studio UI](https://ml.azure.com/).

![AML Hyperdrive Metrics](https://tsmatz.github.io/images/github/azure-ml-tensorflow-complete-sample/20220225_Hyperdrive_Metrics.jpg)

## Remove AML compute

**You don't need to remove your AML compute** for saving money, because the nodes will be automatically terminated, when it's inactive.<br>
But if you want to clean up, please run as follows.

In [8]:
!az ml compute delete --name hypertest01 \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --yes

Command group 'ml compute' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Deleting compute hypertest01 
.......Done.
(0m 37s)

